### Information Security Project

In [3]:
# ! .\isProjectEnv\Scripts\activate
# .\isProjectEnv\Scripts\activate
#imports
import numpy as np
from sympy import Matrix

In [5]:
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
char_to_num = {char: i for i, char in enumerate(alphabet)}
num_to_char = {i: char for i, char in enumerate(alphabet)}

def getKeyMatrix(key, n):
    """
    Generates the key matrix for the Hill cipher.
    """
    k = 0
    keyMatrix = [[0] * n for i in range(n)]
    for i in range(n):
        for j in range(n):
            keyMatrix[i][j] = ord(key[k]) % 65
            k += 1
    return np.array(keyMatrix)

def padding(plaintext, n):
    """
    Pads the plaintext with 'X' to make its length a multiple of n.
    Returns padded plaintext and number of padding chars added.
    """
    plaintext = plaintext.replace(" ", "").upper()
    pad_len = (n - len(plaintext) % n) % n
    plaintext += 'X' * pad_len
    return plaintext, pad_len

def encrypt(plaintext_blocks, key_matrix):
    """
    Encrypts plaintext blocks using the Hill cipher.
    """
    ciphertext = ""
    for block in plaintext_blocks:
        encrypted_block = np.dot(key_matrix, block) % 26
        for num in encrypted_block:
            ciphertext += num_to_char[num[0]]
    return ciphertext

def modMatInv(matrix, modulus):
    """
    Computes modular inverse of a matrix under modulus.
    Uses sympy for adjugate + determinant.
    """
    M = Matrix(matrix)
    det = int(M.det()) % modulus
    if np.gcd(det, modulus) != 1:
        raise ValueError("Key matrix not invertible modulo {}".format(modulus))
    det_inv = pow(det, -1, modulus)
    adjugate = M.adjugate()
    return np.array((det_inv * adjugate) % modulus).astype(int)

def decrypt(cipher_blocks, inverse_key_matrix, pad_len=0):
    """
    Decrypts ciphertext blocks using the Hill cipher.
    Removes padding automatically.
    """
    decrypted_text = ""
    for block in cipher_blocks:
        decrypted_block = np.dot(inverse_key_matrix, block) % 26
        for num in decrypted_block:
            decrypted_text += num_to_char[num[0]]
    if pad_len > 0:
        decrypted_text = decrypted_text[:-pad_len]  # strip padding
    return decrypted_text


# ------------------------------
# Example run
# ------------------------------

# Key matrix
key_matrix = getKeyMatrix("GYBNQKURP", 3)
print("Key Matrix:\n", key_matrix)

# Plaintext
plaintext = "Quantifying Confusion and Diffusion"
plaintext, pad_len = padding(plaintext, 3)
plaintext_block = np.array([[char_to_num[char]] for char in plaintext])
plaintext_blocks = [plaintext_block[i:i+3] for i in range(0, len(plaintext_block), 3)]
print("\nPadded Plaintext:", plaintext)

# Encryption
ciphertext = encrypt(plaintext_blocks, key_matrix)
print("Ciphertext:", ciphertext)

# Ciphertext to numeric blocks
encrypted_block = np.array([[char_to_num[char]] for char in ciphertext])
cipher_blocks = [encrypted_block[i:i+3] for i in range(0, len(encrypted_block), 3)]

# Inverse key matrix
inverse_key_matrix = modMatInv(key_matrix, 26)
print("\nInverse Key Matrix:\n", inverse_key_matrix)

# Decryption (auto removes padding)
decrypted_text = decrypt(cipher_blocks, inverse_key_matrix, pad_len)
print("Decrypted Text:", decrypted_text)


Key Matrix:
 [[ 6 24  1]
 [13 16 10]
 [20 17 15]]

Padded Plaintext: QUANTIFYINGCONFUSIONANDDIFFUSIONX
Ciphertext: EIKWHBQJEQZCLYEOEUGAHXNSRAIOEUDWO

Inverse Key Matrix:
 [[ 8  5 10]
 [21  8 21]
 [21 12  8]]
Decrypted Text: QUANTIFYINGCONFUSIONANDDIFFUSION


In [7]:
class HillCipher:
    def __init__(self, key, n):
        self.alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        self.char_to_num = {char: i for i, char in enumerate(self.alphabet)}
        self.num_to_char = {i: char for i, char in enumerate(self.alphabet)}
        self.n = n
        self.key_matrix = self.getKeyMatrix(key, n)
        self.inverse_key_matrix = self.modMatInv(self.key_matrix, 26)

    def getKeyMatrix(self, key, n):
        """
        Generates the key matrix for the Hill cipher.
        """
        k = 0
        keyMatrix = [[0] * n for _ in range(n)]
        for i in range(n):
            for j in range(n):
                keyMatrix[i][j] = ord(key[k]) % 65
                k += 1
        return np.array(keyMatrix)

    def padding(self, plaintext):
        """
        Pads the plaintext with 'X' to make its length a multiple of n.
        Returns padded plaintext and number of padding chars added.
        """
        plaintext = plaintext.replace(" ", "").upper()
        pad_len = (self.n - len(plaintext) % self.n) % self.n
        plaintext += 'X' * pad_len
        return plaintext, pad_len

    def encrypt(self, plaintext):
        """
        Encrypts plaintext blocks using the Hill cipher.
        """
        plaintext, pad_len = self.padding(plaintext)
        blocks = np.array([[self.char_to_num[char]] for char in plaintext])
        plaintext_blocks = [blocks[i:i+self.n] for i in range(0, len(blocks), self.n)]
        ciphertext = ""
        for block in plaintext_blocks:
            encrypted_block = np.dot(self.key_matrix, block) % 26
            for num in encrypted_block:
                ciphertext += self.num_to_char[num[0]]
        return ciphertext, pad_len

    def modMatInv(self, matrix, modulus):
        """
        Computes modular inverse of a matrix under modulus.
        Uses sympy for adjugate + determinant.
        """
        M = Matrix(matrix)
        det = int(M.det()) % modulus
        if np.gcd(det, modulus) != 1:
            raise ValueError("Key matrix not invertible modulo {}".format(modulus))
        det_inv = pow(det, -1, modulus)
        adjugate = M.adjugate()
        return np.array((det_inv * adjugate) % modulus).astype(int)

    def decrypt(self, ciphertext, pad_len=0):
        """
        Decrypts ciphertext blocks using the Hill cipher.
        Removes padding automatically.
        """
        blocks = np.array([[self.char_to_num[char]] for char in ciphertext])
        cipher_blocks = [blocks[i:i+self.n] for i in range(0, len(blocks), self.n)]
        decrypted_text = ""
        for block in cipher_blocks:
            decrypted_block = np.dot(self.inverse_key_matrix, block) % 26
            for num in decrypted_block:
                decrypted_text += self.num_to_char[num[0]]
        if pad_len > 0:
            decrypted_text = decrypted_text[:-pad_len]
        return decrypted_text
    def getCiphertext(self, plaintext):
        """
        Encrypts the given plaintext and returns the ciphertext and padding length.
        """
        ciphertext, pad_len = self.encrypt(plaintext)
        return ciphertext, pad_len

In [ ]:
# experimenting avalanche effect make a small change to the plaintext (an "incremental change") and re-encrypt it. The goal is to measure the 
#avalanche effect, which is the fraction of ciphertext symbols that change due to this small plaintext modification.  
# Then repeat this experiment with multiple positions and plaintexts and then calculate the average result

hilCipherObject = HillCipher("GYBNQKURP", 3)
ciphertext, pad_len = hilCipherObject.getCiphertext("Quantifying Confusion and Diffusion")
print("Ciphertext from class:", ciphertext)        
import random
def measure_avalanche_effect(hill_cipher, original_plaintext, num_trials=100):
    total_changed_chars = 0
    original_ciphertext, _ = hill_cipher.getCiphertext(original_plaintext)
    
    for _ in range(num_trials):
        # Randomly change one character in the plaintext
        pos = random.randint(0, len(original_plaintext) - 1)
        modified_plaintext = list(original_plaintext)
        modified_plaintext[pos] = random.choice(hill_cipher.alphabet.replace(modified_plaintext[pos], ''))
        modified_plaintext = ''.join(modified_plaintext)        

        # Encrypt the modified plaintext
        modified_ciphertext, _ = hill_cipher.getCiphertext(modified_plaintext)
        
        # Count the number of differing characters in the ciphertexts
        changed_chars = sum(1 for a, b in zip(original_ciphertext, modified_ciphertext) if a != b)
        total_changed_chars += changed_chars
    
    average_changed_chars = total_changed_chars / num_trials
    avalanche_effect = average_changed_chars / len(original_ciphertext)
    return avalanche_effect
avalanche_effect = measure_avalanche_effect(hilCipherObject, "Quantifying Confusion and Diffusion")
print(f"Avalanche Effect: {avalanche_effect:.2%}")

Ciphertext from class: EIKWHBQJEQZCLYEOEUGAHXNSRAIOEUDWO
Avalanche Effect: 12.36%


In [10]:
#To quantify confusion, you'll encrypt a fixed plaintext using a specific key. Then, make an incremental change to the key and re-encrypt the same plaintext. 
# You'll then measure the resulting change in the ciphertext and report the average changes. This process helps measure the cipher's 
#confusion property by observing how sensitive the ciphertext is to small changes in the key.
def measure_confusion(hill_cipher, original_plaintext, num_trials=100):
    total_changed_chars = 0
    original_ciphertext, _ = hill_cipher.getCiphertext(original_plaintext)
    
    for _ in range(num_trials):
        # Randomly change one character in the key
        pos = random.randint(0, len(hill_cipher.alphabet) - 1)
        modified_key = list(hill_cipher.alphabet)
        modified_key[pos] = random.choice(hill_cipher.alphabet.replace(modified_key[pos], ''))
        modified_key = ''.join(modified_key)[:hill_cipher.n * hill_cipher.n]  # Ensure key length matches
        
        # use getKeyMatrix to get the key matrix and check if it is invertible mod 26
        try:
            modified_key_matrix = hill_cipher.getKeyMatrix(modified_key, hill_cipher.n)
            _ = hill_cipher.modMatInv(modified_key_matrix, 26)  # Check invertibility
        except ValueError:
            continue  # Skip if not invertible
        

        # Create a new HillCipher object with the modified key
        modified_hill_cipher = HillCipher(modified_key, hill_cipher.n)
        
        # Encrypt the same plaintext with the modified key
        modified_ciphertext, _ = modified_hill_cipher.getCiphertext(original_plaintext)
        
        # Count the number of differing characters in the ciphertexts
        changed_chars = sum(1 for a, b in zip(original_ciphertext, modified_ciphertext) if a != b)
        total_changed_chars += changed_chars
    
    average_changed_chars = total_changed_chars / num_trials
    confusion_effect = average_changed_chars / len(original_ciphertext)
    return confusion_effect

confusion_effect = measure_confusion(hilCipherObject, "Quantifying Confusion and Diffusion")
print(f"Confusion Effect: {confusion_effect:.2%}")

Confusion Effect: 6.85%


In [1]:
import secretpy as spy

In [2]:
import pycipher as pc